In [1]:
%matplotlib inline
import csv
import requests # pip install requests for easy http request for CSV data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn import linear_model, datasets, preprocessing, metrics


1. Load the Galton dataset into a Pandas dataframe?

In [2]:
df=pd.read_csv("http://www.randomservices.org/random/data/Galton.txt", sep="\t")

2. Summarize the dataset:
    * Number of rows
    * Average height of male/female kids
    * Std deviation of male/female kids

In [3]:
male = df['Gender'] == "M"
female = df['Gender'] == "F"

male_avg = np.average(df[male]['Height'])
female_avg = np.average(df[female]['Height'])

print("male avg: " + str(male_avg))
print("female avg: " + str(female_avg))

male avg: 69.22881720430108
female avg: 64.11016166281756


In [4]:
male = df['Gender'] == "M"
female = df['Gender'] == "F"

male_std = np.std(df[male]['Height'])
female_std = np.std(df[female]['Height'])

print("male std: " + str(male_std))
print("female std: " + str(female_std))

male std: 2.6287630884482325
female std: 2.367580931200028


Create a training and test dataset. The test dataset should be at least 25%.

In [5]:
df

X_father = df[['Father']]
X_mother = df[['Mother']]

y = df['Height']
           
X_train_mother, X_test_mother, y_train_mother, y_test_mother = train_test_split(X_mother, y, test_size=0.25, random_state=42)
X_train_father, X_test_father, y_train_father, y_test_father = train_test_split(X_father, y, test_size=0.25, random_state=42)

#lab_enc = preprocessing.LabelEncoder()
#y_train_father = lab_enc.fit_transform(y_train_father)

#lab_enc = preprocessing.LabelEncoder()
#y_train_mother = lab_enc.fit_transform(y_train_mother)

4. Create 2 regression models: for predicting the childs height based on (i) father height and (ii) mother's height!

In [6]:
model_father = linear_model.LinearRegression()
model_father.fit(X_train_father, y_train_father)

model_mother = linear_model.LinearRegression()
model_mother.fit(X_train_mother, y_train_mother)

pred_father = model_father.predict(X_test_father)
pred_mother = model_mother.predict(X_test_mother)


5. Compute the model quality parameters: $R^{2}$ and $MSE$! 

In [7]:
mse_father = metrics.mean_squared_error(y_test_father,pred_father)
mse_mother = metrics.mean_squared_error(y_test_mother,pred_mother)

r_squared_father = metrics.r2_score(y_test_father,pred_father)
r_squared_mother = metrics.r2_score(y_test_mother,pred_mother)

print("MSE Father: " + str(mse_father) + " MSE Mother: " + str(mse_mother))
print("R^2 Father: " + str(r_squared_father) + " R^2 Mother: " + str(r_squared_mother))

MSE Father: 11.56529244006722 MSE Mother: 11.439403452832908
R^2 Father: 0.041313375962950705 R^2 Mother: 0.05174874444157118


6. Create a multi-variate regression model including both the mother and father height as features! How does the $R^{2}$ change?

In [8]:
X_all = df[['Father','Mother']]
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y, test_size=0.25, random_state=42)

model_all = linear_model.LinearRegression()
model_all.fit(X_train_all, y_train_all)

pred_all = model_all.predict(X_test_all)
r_squared_all = metrics.r2_score(y_test_all, pred_all)
mse_all = metrics.mean_squared_error(pred_all,y_test_all)
#print("MSE Father: " + str(mse_father) + " MSE Mother: " + str(mse_mother))
print("R^2 All: " + str(r_squared_all) + " MSE All: " + str(mse_all))

R^2 All: 0.07640810797419273 MSE All: 11.141920684751984


7. Create a Spark MLlib model for the same task!

In [9]:
# Initialize PySpark
import os, sys
os.environ["JAVA_HOME"]="/lrz/sys/compilers/java/jdk1.8.0_112"
APP_NAME = "PySpark Lecture"
SPARK_MASTER="local[16]"
import pyspark
import pyspark.sql
from pyspark.sql import Row
conf=pyspark.SparkConf()
conf=pyspark.SparkConf().setAppName(APP_NAME).set("spark.local.dir", os.path.join(os.getcwd(), "tmp"))
sc = pyspark.SparkContext(master=SPARK_MASTER, conf=conf)
spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initiated...")


from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression


spark_df = spark.createDataFrame(df)
#dataset = iris_df

#dataset = spark.createDataFrame(
#    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
 #   ["id", "hour", "mobile", "userFeatures", "clicked"])

assembler = VectorAssembler(
    inputCols=["Father", "Mother"],
    outputCol="target2")

output = assembler.transform(spark_df)
output.select("target2", "Height").show(truncate=False)


PySpark initiated...
+-----------+------+
|target2    |Height|
+-----------+------+
|[78.5,67.0]|73.2  |
|[78.5,67.0]|69.2  |
|[78.5,67.0]|69.0  |
|[78.5,67.0]|69.0  |
|[75.5,66.5]|73.5  |
|[75.5,66.5]|72.5  |
|[75.5,66.5]|65.5  |
|[75.5,66.5]|65.5  |
|[75.0,64.0]|71.0  |
|[75.0,64.0]|68.0  |
|[75.0,64.0]|70.5  |
|[75.0,64.0]|68.5  |
|[75.0,64.0]|67.0  |
|[75.0,64.0]|64.5  |
|[75.0,64.0]|63.0  |
|[75.0,58.5]|72.0  |
|[75.0,58.5]|69.0  |
|[75.0,58.5]|68.0  |
|[75.0,58.5]|66.5  |
|[75.0,58.5]|62.5  |
+-----------+------+
only showing top 20 rows



In [10]:
output.sort("Height").show()

+------+------+------+------+------+----+-----------+
|Family|Father|Mother|Gender|Height|Kids|    target2|
+------+------+------+------+------+----+-----------+
|   155|  68.0|  60.0|     F|  56.0|   7|[68.0,60.0]|
|   185|  66.0|  59.0|     F|  57.0|  15|[66.0,59.0]|
|   204|  62.5|  63.0|     F|  57.0|   2|[62.5,63.0]|
|   176|  66.5|  65.0|     F|  57.5|   8|[66.5,65.0]|
|   185|  66.0|  59.0|     F|  58.0|  15|[66.0,59.0]|
|   190|  65.0|  65.0|     F|  59.0|   9|[65.0,65.0]|
|   109|  69.5|  64.5|     M|  60.0|   7|[69.5,64.5]|
|   155|  68.0|  60.0|     F|  60.0|   7|[68.0,60.0]|
|   188|  65.0|  66.0|     F|  60.0|   4|[65.0,66.0]|
|    64|  70.0|  67.0|     F|  60.0|   5|[70.0,67.0]|
|   109|  69.5|  64.5|     F|  60.0|   7|[69.5,64.5]|
|   164|  67.0|  65.5|     F|  60.0|   4|[67.0,65.5]|
|   183|  66.0|  60.0|     F|  60.0|   4|[66.0,60.0]|
|   155|  68.0|  60.0|     F|  60.0|   7|[68.0,60.0]|
|    56|  71.0|  62.0|     F|  60.0|   5|[71.0,62.0]|
|   173|  66.0|  67.0|     F